<a href="https://colab.research.google.com/github/maxseats/2023_data_public/blob/main/whisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델 테스트 해보기
- 데이터셋 불러와서 평가지표(CER, WER)로 모델 점수 출력
- 파일로 저장
    - total.json : 모델 전체 평가
    - 각 모델 폴더 : 해당 모델 평가 과정 로그 기록

## 필요 라이브러리 설치 및 import

In [ ]:
from google.colab import drive

# 구글드라이브 마운트
drive.mount('/content/drive')

!pip install nlptutti

import nlptutti as metrics
from transformers import pipeline
import json
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 사용자 설정 변수

In [ ]:
#----- 모델 이름 설정 -----

# model_name = "openai/whisper-large-v3"

model_name = "openai/whisper-large-v3"
model_name = "SungBeom/whisper-small-ko"
model_name = "seastar105/whisper-medium-ko-zeroth"

model_names = ["SungBeom/whisper-small-ko", "openai/whisper-large-v3", "seastar105/whisper-medium-ko-zeroth"]

#----- 모델 이름 설정 -----


data_num = 75   # 데이터 개수
data_directory = "/content/drive/MyDrive/STT_test/discord_dataset" # 데이터셋 폴더
test_log_path = "/content/drive/MyDrive/STT_test/test_log"    # 테스트 결과 및 로그 저장위치

## 테스트 시작

In [ ]:
# 평균 계산용
CER_total = 0.0
WER_total = 0.0

# 모델 별 테스트 파이프라인 실행
for model_name in model_names:

    start_time = time.time()    # 시작 시간 기록

    # 모델 폴더 생성 및 로그파일 폴더 지정
    #model_log_dir = test_log_path + "/" + model_name
    model_log_dir = os.path.join(test_log_path, model_name)
    os.makedirs(model_log_dir, exist_ok=True)
    log_file_path = os.path.join(model_log_dir, "log.txt")


    with open(log_file_path, 'w', encoding='utf-8') as log_file:

        pipe = pipeline("automatic-speech-recognition", model=model_name)   # STT 파이프라인

        for ii in range(data_num):

            i=ii+1   # 현재 파일 번호
            print(i, "번째 데이터:")
            log_file.write(f"{i} 번째 데이터:\n")

            sample = data_directory + "/" + "{:03d}".format(i) + ".mp3"    # 음성파일 경로

            result = pipe(sample, return_timestamps=False)


            preds = result["text"]  # STT 예측 문자열
            target_path = data_directory + "/" + "{:03d}".format(i) + ".txt" # 텍스트파일 경로


            # 파일 열기
            with open(target_path, 'r', encoding='utf-8') as file:
                # 파일 내용 읽기
                target = file.read()

            print("예측 : ", result["text"])
            print("정답 : ", target)
            log_file.write(f"예측 : {preds}\n")
            log_file.write(f"정답 : {target}\n")

            # CER 출력
            cer_result = metrics.get_cer(target, preds)

            cer_substitutions = cer_result['substitutions']
            cer_deletions = cer_result['deletions']
            cer_insertions = cer_result['insertions']
            # prints: [cer, substitutions, deletions, insertions] -> [CER = 0 / 34, S = 0, D = 0, I = 0]
            CER_total += cer_result['cer']
            print("CER, S, D, I : ", cer_result['cer'], cer_substitutions, cer_deletions, cer_insertions)
            log_file.write(f"CER, S, D, I : {cer_result['cer']}, {cer_substitutions}, {cer_deletions}, {cer_insertions}\n")


            # WER 출력
            wer_result = metrics.get_wer(target, preds)

            wer_substitutions = wer_result['substitutions']
            wer_deletions = wer_result['deletions']
            wer_insertions = wer_result['insertions']
            # prints: [wer, substitutions, deletions, insertions] -> [WER =  2 / 4, S = 1, D = 1, I = 0]
            WER_total += wer_result['wer']
            print("WER, S, D, I : ", wer_result['wer'], wer_substitutions, wer_deletions, wer_insertions)
            print()
            log_file.write(f"WER, S, D, I : {wer_result['wer']}, {wer_substitutions}, {wer_deletions}, {wer_insertions}\n\n")


    end_time = time.time()  # 종료 시간 기록
    elapsed_time = end_time - start_time    # 실행 시간

    # 시간, 분, 초 단위로 변환
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)


    print("현재 모델 : ", model_name)
    print("CER 평균 : ", CER_total / data_num)
    print("WER 평균 : ", WER_total / data_num)
    print("실행시간 : ", "{:02d}시간 {:02d}분 {:02d}초".format(hours, minutes, seconds))

    # 데이터 딕셔너리 생성
    data = {
        "model_name": model_name,
        "CER_mean": CER_total / data_num,
        "WER_mean": WER_total / data_num
        "running_time" : "{:02d}:{:02d}:{:02d}".format(hours, minutes, seconds)
    }

    # JSON 파일에 저장(내용 추가)
    with open(test_log_path + "/total_result.json", "a", encoding="utf-8") as file:
        json.dump(data, file, ensure_ascii=False, indent=4)
